# K-MCI
### Intro
In this notebook we have the main algorithm of the article. All necessary functions are implemented in function.py file.<br>
This document have blow sections:<br>
1. Importing Libraries & classes
2. Reading datasets
3. Preprocessing datasets
4. Main Algorithm

### Importing Libraries & classes
We import some libraries from third party.

In [38]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import random
import copy
from sklearn import preprocessing

In [39]:
# defnition of Candidate class
class Candidate:    
    def __init__(self,nodes,sampling_interval,fitness=-1):
        self.nodes = nodes
        self.sampling_interval = sampling_interval 
        self.variation = []
        self.random_number
        self.fitness = fitness
        self.centers = []
        self.probability = []
        self.labels = []
        self.each_fitness = []
        
    def random_number(self):
        self.random_number = random.random()
        
    def describe(self):
        print("Candidate with #{} nodes, #{} centers, fitness={}\n clusters:\n{} \n Centers:\n{}\n sampling_interval:\n{}"
              .format(len(self.nodes),len(self.centers),self.fitness,self.nodes[np.random.randint(self.nodes.shape[0], size=5), :],self.centers,self.sampling_interval))

### Reading Datasets and Preprocessing
We show the structure of our data for all of datasets.<br>
We have these datasets from UCI Machine Learning Repository:
1. Breast Cancer Wisconsin
2. Contraceptive Method Choice data
3. Glass
4. Iris
5. Vowel
6. Wine

In [40]:
""" In this cell we have some code which will do preprocessing for us on all datasets """
""" the resourses of these datasets are mentioned in README.md file. """

path = 'datasets/' 
# because we are using local files, you need to download datasets and change the "path" variable
# local folder of your downloaded datasets


def bcw(): 
    # importing dataset
    dataset = pd.read_csv(path+'bcw.csv',
                          names = ['Sample code number','Clump Thickness','Uniformity of Cell Size',
                                   'Uniformity of Cell Shape','Marginal Adhesion','Single Epithelial',
                                   'Bare Nuclei','Bland Chromatin','Normal Nucleoli','Mitoses','Class'])
    dataset = dataset.drop(dataset.columns[0], axis=1)
    dataset= dataset.replace(to_replace='?',value=np.nan)
    dataset = dataset.dropna(axis =0) # resolving missing values
    dataset = dataset.astype('int64') # casting string valued column to int64
    x = dataset.iloc[:,:-1].values # features
    y = dataset.iloc[:,-1].values # target values
    return (x,y,dataset)


def cmc():
    # importing dataset
    dataset = pd.read_csv(path+'cmc.csv', names=["Wife's age","Wife's education","Husband's education",
                                                 "Number of children ever born","Wife's religion",
                                                 "Wife's now working?","Husband's occupation",
                                                 "Standard-of-living index","Media exposure","Contraceptive method used"])
    x = dataset.iloc[:,:-1].values # features
    y = dataset.iloc[:,-1].values # target values
    return (x,y,dataset)


def glass():
    # importing dataset
    dataset = pd.read_csv(path+'glass.csv', names= ['Id','refractive index','Sodium','Magnesium','Aluminum','Silicon','Potassium'
                                                    ,'Calcium','Barium','Iron','glass'])
    x = dataset.iloc[:,1:10].values # features
    y = dataset.iloc[:,-1].values # target values
    return (x,y,dataset)


def iris():
    # importing dataset
    dataset = pd.read_csv(path+'iris.csv', names= ['sepal length','sepal width','petal length','petal width','class'])
    x = dataset.iloc[:,:-1].values # features
    y = dataset.iloc[:,-1].values # target values
    
    # encoding categorial data types to labelEncoder
    from sklearn.preprocessing import LabelEncoder
    labelencoder_y = LabelEncoder()
    labelencoder_y = labelencoder_y.fit(y)
    y = labelencoder_y.transform(y)  # 0 for 'Iris-setosa', 1 for 'Iris-versicolor', 2 for 'Iris-virginica'
    return (x,y,dataset)


def vowel():
    # importing dataset
    dataset = pd.read_csv(path+'vowel.csv', names= ['vowel','type 1 frq','type 2 frq','type 3 frq'])
    x = dataset.iloc[:,1:].values # features
    y = dataset.iloc[:,0].values # target values
    return (x,y,dataset)

def wine():
    # importing dataset
    dataset = pd.read_csv(path+'wine.csv', names= ['class','Alcohol','Malic acid','Ash','Alcalinity of ash','Magnesium','Total phenols','Flavanoids','Nonflavanoid phenols','Proanthocyanins','Color intensity','Hue','OD280/OD315','Proline'])
    x = dataset.iloc[:,1:].values # features
    y = dataset.iloc[:,0].values # target values
    return (x,y,dataset)

In [41]:
# Getting data from our preprocessing class Datasets
x_bcw, y_bcw, df_bcw = bcw()
x_cmc, y_cmc, df_cmc = cmc()
x_glass, y_glass, df_glass = glass()
x_iris, y_iris, df_iris = iris()
x_vowel, y_vowel, df_vowel = vowel()
x_wine, y_wine, df_wine = wine()

In [42]:
# Showing dataset of Breast Cancer Wisconsin
df_bcw.head(3)

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2


In [43]:
# Showing dataset of Contraceptive Method Choice data
df_cmc.head(3)

,Wife's age,Wife's education,Husband's education,Number of children ever born,Wife's religion,Wife's now working?,Husband's occupation,Standard-of-living index,Media exposure,Contraceptive method used
0,24,2,3,3,1,1,2,3,0,1
1,45,1,3,10,1,1,3,4,0,1
2,43,2,3,7,1,1,3,4,0,1


In [44]:
# Showing dataset of Glass
df_glass.head(3)

,Id,refractive index,Sodium,Magnesium,Aluminum,Silicon,Potassium,Calcium,Barium,Iron,glass
0,1,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,2,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,3,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1


In [45]:
# Showing dataset of Iris
df_iris.head(3)

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa


In [46]:
# Showing dataset of Vowel
df_vowel.head(3)

,vowel,type 1 frq,type 2 frq,type 3 frq
0,1,700,1500,2600
1,1,550,1550,2400
2,1,700,1500,2600


In [47]:
# Showing dataset of Wine
df_wine.head(3)

,class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185


### Standardization
In this cell, we standard our data using this equation: <img src='http://3.bp.blogspot.com/_xqXlcaQiGRk/RpO4yR0oKtI/AAAAAAAAABM/7rUWCMwizus/s200/fig2.png'></img>

In [48]:
x_bcw = preprocessing.scale(x_bcw)
x_wine = preprocessing.scale(x_wine)
x_cmc = preprocessing.scale(x_cmc)
#x_glass = preprocessing.scale(x_glass)
x_iris = preprocessing.scale(x_iris)
x_vowel = preprocessing.scale(x_vowel)

C:\Users\HaMeD\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


# Main Algorithm 

The implementation of main algorithm have below sections:
1. <Strong>Initializing Parameters and Candidates</Strong>
    1. Sampling Interval
    2. Apply Random Centers to Candidates
    3. Visualizing Candidates
2. <Strong>Running Kmean to Exploit</Strong>
    1. K-means Algorithm
    2. Visualizing Candidates
3. <Strong>Mutation Algorithm</Strong>
    1. Fitness over Mutated Candidate
    2. Probablility Equation
    3. Mutation Logic

**Extra details will be explain in other next sections**


## Initializing Parameters and Candidates
In this step, we initialize our algorithm. There are some parameters which are important to convergance speed and quality of solution. So we initialize them here based on the paper.<br>
Each time you want to run K-MCI algorithm, you should first do this initialization.<br>

In [49]:
# notice: this values related to IRIS dataset
candidates_array = [] # the array of all candidates
number_of_candidates = 5
sampling_interval_reduction_factor = 0.95
convergence_parameter = None # what is this in paper???
mutation_random = 0.7
iterations_count = 3500
variations_count = 15
number_of_clusters = 6 # in this article number of clusters are predefined

## Sampling Interval
this function use for making sampling_interval


In [50]:
def sampling_intervals(input_array):
    Maxes = np.amax(input_array,axis=0)
    Mins  = np.amin(input_array,axis=0)
    return (Maxes,Mins)

In [51]:
# initializing candidates
for i in range(0 ,number_of_candidates):
    candidates_array.append(Candidate(x_iris,sampling_interval = sampling_intervals(x_iris)))

## Visualizing Candidates
some parameters are uninitialized. They will have value after running Kmean and Cohort and mutations.

In [52]:
candidates_array[random.randint(0,number_of_candidates-1)].describe()

Candidate with #150 nodes, #0 centers, fitness=-1
 clusters:
[[ 1.03800476  0.09821729  0.36489628  0.26414192]
 [ 0.4321654  -0.36217625  0.30805885  0.13250973]
 [ 1.2803405   0.09821729  0.76275827  1.44883158]
 [-0.53717756  1.47939788 -1.2833891  -1.3154443 ]
 [-0.90068117  1.01900435 -1.34022653 -1.18381211]] 
 Centers:
[]
 sampling_interval:
(array([2.4920192 , 3.09077525, 1.78583195, 1.71209594]), array([-1.87002413, -2.43394714, -1.56757623, -1.44707648]))


## Apply Random Centers to Candidates
In this step, we just select some random nodes as centers with psuedorandom functions in python native library.

In [53]:
for c in range(0,number_of_candidates):
    candidates_array[c].centers =  np.asarray(random.sample(list(candidates_array[c].nodes),number_of_clusters))
    print(candidates_array[c].centers)

[[-1.14301691  1.24920112 -1.34022653 -1.44707648]
 [ 0.18982966 -0.36217625  0.42173371  0.3957741 ]
 [ 0.55333328 -0.82256978  0.64908342  0.79067065]
 [-0.17367395 -1.28296331  0.70592084  1.05393502]
 [-1.74885626  0.32841405 -1.39706395 -1.3154443 ]
 [ 0.67450115  0.32841405  0.87643312  1.44883158]]
[[-0.29484182 -0.13197948  0.194384    0.13250973]
 [-0.05250608 -0.59237301  0.76275827  1.58046376]
 [-0.90068117  1.47939788 -1.2833891  -1.05217993]
 [-0.90068117  0.55861082 -1.16971425 -0.92054774]
 [ 0.55333328  0.78880759  1.0469454   1.58046376]
 [ 0.18982966  0.78880759  0.42173371  0.52740629]]
[[-0.29484182 -0.36217625 -0.08980313  0.13250973]
 [ 0.31099753 -0.36217625  0.53540856  0.26414192]
 [ 1.03800476  0.09821729  0.53540856  0.3957741 ]
 [ 2.24968346  1.70959465  1.6721571   1.31719939]
 [-0.7795133   2.40018495 -1.2833891  -1.44707648]
 [-0.41600969  2.63038172 -1.34022653 -1.3154443 ]]
[[ 6.74501145e-01  9.82172869e-02  9.90107977e-01  7.90670654e-01]
 [ 7.9566901

## K-means Algorithm 
this function fitting k-means model on dataset

In [54]:
def doKmeans(x,clusters_count,init_centers):
    kmeans = KMeans(n_clusters = clusters_count, init = init_centers,n_init = 50)
    kmeans = kmeans.fit(x)
    labels = kmeans.labels_
    return (labels,kmeans.cluster_centers_,kmeans.inertia_)


## Running Kmean to Exploit
Here by running Kmean on all of our candidates, they centers initialized and nodes assigns to clusters based on Kmean++ algorithm.<br>
The objective function of Kmean is also the object function of K-MCI. <br>
So the fitness of any candidates is calculated using Kmean inertia or mean square error of nodes to their centers distances.

In [55]:

for c in range(0,number_of_candidates):
    y_predict_temp,centers_temp,fitness_temp = doKmeans(candidates_array[c].nodes,clusters_count = number_of_clusters,init_centers = candidates_array[c].centers)
    candidates_array[c].centers = centers_temp
    candidates_array[c].fitness = fitness_temp
    candidates_array[c].labels   = y_predict_temp
    
candidates_array[2].labels

C:\Users\HaMeD\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=50
  return_n_iter=True)


array([4, 4, 4, 4, 4, 5, 4, 4, 4, 4, 5, 4, 4, 4, 5, 5, 5, 4, 5, 5, 4, 5,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4,
       5, 4, 5, 4, 5, 4, 2, 1, 2, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 2, 1, 2, 1, 2, 2, 0, 2, 1, 3,
       2, 1, 2, 1, 1, 2, 2, 3, 2, 0, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 3,
       2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1])

## Visualizing Candidates
In this step, by Kmeans, we assigned centers to each candidates and calculated the fitness of each candidate using inertia

In [56]:
candidates_array[random.randint(0,number_of_candidates-1)].describe()

Candidate with #150 nodes, #6 centers, fitness=82.05513588130117
 clusters:
[[-0.90068117  1.70959465 -1.22655167 -1.3154443 ]
 [ 0.67450115 -0.36217625  0.30805885  0.13250973]
 [ 0.55333328 -1.74335684  0.36489628  0.13250973]
 [ 0.55333328 -0.59237301  0.76275827  0.3957741 ]
 [ 0.18982966  0.78880759  0.42173371  0.52740629]] 
 Centers:
[[-0.43620433 -1.09113267  0.09018205  0.00636222]
 [ 0.34465528 -1.05276654  0.75960063  0.79798355]
 [-0.72135272  1.5070215  -1.30157708 -1.22066912]
 [-1.30780521  0.19950386 -1.30839757 -1.28911786]
 [ 1.42480988  0.23102311  1.21964374  1.29188551]
 [ 0.56090627 -0.11039853  0.55317026  0.45747669]]
 sampling_interval:
(array([2.4920192 , 3.09077525, 1.78583195, 1.71209594]), array([-1.87002413, -2.43394714, -1.56757623, -1.44707648]))


In [57]:
sum_of_squares(candidates_array[0].nodes,candidates_array[0].centers,candidates_array[0].labels)

83.07070771360613

## Fitness over Mutated Candidate
Just simple sum of square of distances

In [58]:
def sum_of_squares(data, centroids, labels):
    sqe = 0
    for l in np.unique(labels):
        data_l = data[labels == l]
        resid = data_l - centroids[l]
        sqe += (resid**2).sum()
    return sqe

### Mean Square Distance to a Specific Center
This function is exactly based on "sum_of_squares" function. But here all centers not engaged because we want to find the sum of square distances of nodes of a cluster to their cluster center.<br>
The idea is when mutation process done, at least one dimension may get better center but the other goes bad. So we want to have that center which is better to the previous time.

In [59]:
def center_dist(data, center, labels, center_position, center_cluster):
    sqe = 0
    for l in range(0,len(labels)):
        if center_cluster == labels[l] :
            data_l = data[l][center_position]
            resid = data_l - center
            sqe += (resid**2)
    return sqe

In [60]:
def each_fitness(candidates_array_example):
    fitness_of_each_centers = []
    for i in range(0,len(candidates_array_example.centers)):
        Second_layer = []
        for j in range(0,len(candidates_array_example.centers[i])):
            Second_layer.append(center_dist(candidates_array_example.nodes,candidates_array_example.centers[i][j],candidates_array_example.labels,j,i))
        fitness_of_each_centers.append(Second_layer)
    return np.array(fitness_of_each_centers)
print(each_fitness(candidates_array[0]))

[[ 2.86659271 10.10605512  0.26247756  0.61882257]
 [ 5.62075488  7.36429213  1.87309091  4.53056288]
 [ 5.45957281  3.31672677  2.38116709  2.81327991]
 [ 5.01378444  4.21034016  1.77985486  2.47934075]
 [ 1.59095729  6.03128819  0.21145046  0.25281715]
 [ 6.88732647  5.07531724  1.20820442  1.11663095]]


# test paper own answer
## result:
**paper has wrong answer and numbers**

In [70]:
def just_test(data,center):
    temp1 = np.subtract(center,data)
    temp1 = np.power(temp1, 2)
    sqe = np.sum(temp1)
    return sqe


center_glass  =[ [1.52434 , 12.03344 , 0.01215 , 1.12869 , 71.98256 , 0.19252 , 14.34306 , 0.23039 , 0.15156],
                 [1.51956 , 13.25068 , 0.45229 , 1.53305 , 73.01401 , 0.38472 , 11.15803 , 0.00433 , 0.06599],
                 [1.51362 , 13.15690 , 0.65548 , 3.13123 , 70.50411 , 5.33024 , 6.73773  , 0.67322 , 0.01490],
                 [1.52132 , 13.74692 , 3.51952 , 1.01524 , 71.89517 , 0.21094 , 9.44764  , 0.03588 , 0.04680],
                 [1.51933 , 13.08412 , 3.52765 , 1.36555 , 72.85826 , 0.57913 , 8.36271  , 0.00837 , 0.06182],
                 [1.51567 , 14.65825 , 0.06326 , 2.21016 , 73.25324 , 0.02744 , 8.68548  , 1.02698 , 0.00382] ]

center_iris   =[ [5.01213 , 3.40309 , 1.47163 , 0.23540],
                 [5.93432 , 2.79781 , 4.41787 , 1.41727],
                 [6.73334 , 3.06785 , 5.63008 , 2.10679] ]

center_wine   =[ [13.81262 , 1.83004 , 2.42432 , 17.01717 , 105.41208 , 2.93966 , 3.21965 , 0.34183 , 1.87181 , 5.75329 , 1.05368 , 
                  2.89757  , 1136.97230], 
                 [12.74160 , 2.51921 , 2.41113 , 19.57418 , 98.98807  , 1.97496 , 1.26308 , 0.37480 , 1.46902 , 5.73752 , 1.00197 ,
                  2.38197  , 687.01356 ],
                 [12.50086 , 2.48843 , 2.43785 , 21.43603 , 92.55049  , 2.02977 , 1.54943 , 0.32085 , 1.38624 , 4.38814 , 0.94045 ,
                  2.43190  , 463.86513 ] ]

center_cmc    =[ [43.64742 , 2.99091 , 3.44673 , 4.59136 , 0.80254 , 0.76971 , 1.82586 , 3.42522 , 0.10127 , 1.67635],
                 [24.41296 , 3.03823 , 3.51059 , 1.79036 , 0.92502 , 0.78935 , 2.29463 , 2.97378 , 0.03692 , 2.00149],
                 [33.50648 , 3.13272 , 3.55176 , 3.65914 , 0.79533 , 0.69725 , 2.10130 , 3.28562 , 0.06151 , 2.11479] ]

center_vowel  =[ [506.98650 , 1839.66652 , 2556.20000],
                 [623.71854 , 1309.59677 , 2333.45721],
                 [407.89515 , 1018.05210 , 2317.82688], 
                 [439.24323 , 987.68488  , 2665.47618], 
                 [357.26154 , 2291.44000 , 2977.39697],
                 [375.45357 , 2149.40364 , 2678.44208] ]

center_cancer =[ [7.11701 , 6.64106 , 6.62548 ,5.61469, 5.24061 , 8.10094 , 6.07818 , 6.02147 , 2.32582],
                 [2.88942 , 1.12774 , 1.20072 ,1.16404, 1.99334 , 1.12116 , 2.00537 , 1.10133 , 1.03162] ] 


label = []
for i in x_iris:
    test = []
    for j in center_iris:
        test.append(just_test(i,j))
        print(test)
    label.append(np.argmin(test))
print(label)   
print(sum_of_squares(x_iris,center_iris,label))



[50.9568554664026]
[50.9568554664026, 90.50479355947488]
[50.9568554664026, 90.50479355947488, 122.772908363007]
[60.694204875571955]
[60.694204875571955, 99.29576717561164]
[60.694204875571955, 99.29576717561164, 132.5727675341162]
[61.01593948783875]
[61.01593948783875, 100.95670837772016]
[61.01593948783875, 100.95670837772016, 134.5101193991082]
[63.41023655136755]
[63.41023655136755, 102.62599123809602]
[63.41023655136755, 102.62599123809602, 136.221762443573]
[51.35979553278763]
[51.35979553278763, 91.40988022011086]
[51.35979553278763, 91.40988022011086, 123.7473014578058]
[41.57061861919017]
[41.57061861919017, 79.93575718843584]
[41.57061861919017, 79.93575718843584, 110.34923452431174]
[59.24797731786751]
[59.24797731786751, 99.32350213872772]
[59.24797731786751, 99.32350213872772, 132.50257108277597]
[53.238623872878016]
[53.238623872878016, 92.39646116413724]
[53.238623872878016, 92.39646116413724, 124.84473394631624]
[70.19982030770424]
[70.19982030770424, 109.640113594826

## Mutation algorithm 
this function do mutation algorithm on candidates interval's centers

In [62]:
'''def mutation(candidate_array,mutation_random):
    New_candidate = copy.deepcopy(candidate_array)
    Mutant_candidate = copy.deepcopy(candidate_array)
    Trial_candidate = copy.deepcopy(candidate_array)
    for x in range(len(candidate_array)):
        a = np.full(len(candidate_array), 1/(len(candidate_array)-1))
        a[x] = 0
        temp = np.random.choice(len(candidate_array), 3, replace = False, p=a)
        for i in range(len(candidate_array[0].centers)):
            for j in range(len(candidate_array[0].centers[i])):
                Mutant_candidate[x].centers[i][j] = candidate_array[temp[0]].centers[i][j]+ random.random()*(candidate_array[temp[1]].centers[i][j] - candidate_array[temp[2]].centers[i][j])
                if random.random() < mutation_random:
                    Trial_candidate[x].centers[i][j] = Mutant_candidate[x].centers[i][j] #Trial & Mutant must be copy of same candidate
                    
                    
        Trial_candidate[x].fitness = sum_of_squares(Trial_candidate[x].nodes,Trial_candidate[x].centers,Trial_candidate[x].labels)
        
        if Trial_candidate[x].fitness < sum_of_squares(candidate_array[x].nodes,candidates_array[2].centers,candidates_array[2].labels):
            New_candidate[x] = Trial_candidate[x]
            print("sakjfnr")
        else:
            New_candidate[x] = candidate_array[x]
        #print(Trial_candidate[x].centers)
        #print(candidate_array[x].centers)
        #print(New_candidate[x].centers)
        #print(Mutant_candidate[x].centers)
        print("###########################################################################################################")
        Trial_candidate[x].each_fitness = each_fitness(Trial_candidate[x])
        print(Trial_candidate[x].each_fitness)
        candidate_array[x].each_fitness = each_fitness(candidate_array[x])
        print(candidate_array[x].each_fitness)
        print("###########################################################################################################")
        
    return New_candidate
#print(candidates_array[1].fitness)
#for i in range(0,3500):
New_candidates_array = mutation(candidates_array,mutation_random)
    #if New_candidates_array[1].fitness != candidates_array[1].fitness:
     #   print(New_candidates_array[1].fitness)#never happen'''

'def mutation(candidate_array,mutation_random):\n    New_candidate = copy.deepcopy(candidate_array)\n    Mutant_candidate = copy.deepcopy(candidate_array)\n    Trial_candidate = copy.deepcopy(candidate_array)\n    for x in range(len(candidate_array)):\n        a = np.full(len(candidate_array), 1/(len(candidate_array)-1))\n        a[x] = 0\n        temp = np.random.choice(len(candidate_array), 3, replace = False, p=a)\n        for i in range(len(candidate_array[0].centers)):\n            for j in range(len(candidate_array[0].centers[i])):\n                Mutant_candidate[x].centers[i][j] = candidate_array[temp[0]].centers[i][j]+ random.random()*(candidate_array[temp[1]].centers[i][j] - candidate_array[temp[2]].centers[i][j])\n                if random.random() < mutation_random:\n                    Trial_candidate[x].centers[i][j] = Mutant_candidate[x].centers[i][j] #Trial & Mutant must be copy of same candidate\n                    \n                    \n        Trial_candidate[x].f

In [63]:
def mutation(candidate_array,mutation_random):
    New_candidate = copy.deepcopy(candidate_array)
    Trial_candidate = copy.deepcopy(candidate_array)
    for x in range(len(candidate_array)):
        a = np.full(len(candidate_array), 1/(len(candidate_array)-1))
        a[x] = 0
        temp = np.random.choice(len(candidate_array), 3, replace = False, p=a)
        Centers = []
        for i in range(len(candidate_array[0].centers)):
            Second_layer = []
            for j in range(len(candidate_array[0].centers[i])):
                temporary_Center = candidate_array[temp[0]].centers[i][j]+ random.random()*(candidate_array[temp[1]].centers[i][j] - candidate_array[temp[2]].centers[i][j])
                if random.random() < mutation_random:
                    Second_layer.append(temporary_Center) #Trial & Mutant must be copy of same candidate
                else:
                    Second_layer.append(candidate_array[x].centers[i][j])
            Centers.append(Second_layer)
            
        
        y_predict_temp,centers_temp,fitness_temp = doKmeans(candidate_array[x].nodes,clusters_count = number_of_clusters, init_centers = np.asarray(Centers))
        if fitness_temp < candidates_array[x].fitness:
            New_candidate[x].labels  =  y_predict_temp
            New_candidate[x].fitness = fitness_temp
            New_candidate[x].centers = centers_temp
        New_candidate[x].each_fitness = each_fitness(New_candidate[c])
        print(New_candidate[x].fitness)
        print(candidate_array[x].fitness)
        print("#############################################################################################")
    return New_candidate

New_candidates_array = mutation(candidates_array,mutation_random)

81.93844890989027
83.07070771360611
#############################################################################################
82.05513588130117
82.05513588130117
#############################################################################################
81.93844890989027
82.4725666679249
#############################################################################################
82.05544434964935
98.7933293548069
#############################################################################################
82.85710938496706
82.85710938496706
#############################################################################################


C:\Users\HaMeD\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=50
  return_n_iter=True)


# Probability Equation

In [64]:
def probability(candidates):
    for c in range(0,len(candidates)):
        temp_array = []
        for clusters in range(0,len(candidates[c].centers)):
            temp_array.append((1/np.sum(candidates[c].each_fitness[clusters]))/(np.sum(1/(np.sum(x.each_fitness[clusters])) for x in candidates)))
        print(temp_array)    
        candidates[c].probability = temp_array
    return
probability(New_candidates_array)


[0.19999999999999998, 0.2, 0.2, 0.2, 0.2, 0.19999999999999998]
[0.19999999999999998, 0.2, 0.2, 0.2, 0.2, 0.19999999999999998]
[0.19999999999999998, 0.2, 0.2, 0.2, 0.2, 0.19999999999999998]
[0.19999999999999998, 0.2, 0.2, 0.2, 0.2, 0.19999999999999998]
[0.19999999999999998, 0.2, 0.2, 0.2, 0.2, 0.19999999999999998]


C:\Users\HaMeD\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  """


### Roulette Wheel Selection
This is a logic for selecting targets with higher probability.<br>
In this article, each candidate want to follow the other candidate with higher fitness. So in the previous section <strong><em>Probability Equation<strong><em>, we calculated all probabilities of candidates based on their fitnesses.

In [65]:
def roulette_wheel_selection(inertia_array):
    maximum = np.sum(inertia_array)
    pick = random.uniform(0, maximum)
    current = 0
    for fitness in inertia_array:
        current += fitness
        if current > pick:
            return fitness

# Shrink sampling interval
**here we shrink sampeling interval using reduction factor ro creat neighborhood of each center**

In [66]:
def shrinked_sampling_interval(candidates_array):
    for i in range(0,len(candidates_array)):
        shrinked_sampling_interval = []
        for j in range(0,len(candidates_array[i].centers)):
            Second_layer = []
            for k in range(0,len(candidates_array[i].centers[j])):
                Third_layer = []
                Temp = abs(candidates_array[i].sampling_interval[0][k] - candidates_array[i].sampling_interval[1][k]) * sampling_interval_reduction_factor
                Third_layer.append(candidates_array[i].centers[j][k] - (Temp/2))
                Third_layer.append(candidates_array[i].centers[j][k] + (Temp/2))
                Second_layer.append(Third_layer)
            shrinked_sampling_interval.append(Second_layer)
        candidates_array[i].sampling_interval = shrinked_sampling_interval
    return
shrinked_sampling_interval(New_candidates_array)
print(New_candidates_array[0].sampling_interval)

[[[-2.410432836025704, 1.7335083331471708], [-3.925622187264117, 1.3228640834330205], [-1.4530488190462647, 1.7326889594611916], [-1.441811191926429, 1.5594026098924303]], [[-1.627183526832868, 2.5167576423400067], [-2.9672363170739526, 2.2812499536231847], [-0.9828612049034825, 2.202876573603974], [-0.9457772441258117, 2.0554365576930476]], [[-3.0052738738498443, 1.1386672953230303], [-1.5609701744810685, 3.687516096216069], [-2.876257988962887, 0.30947978954456934], [-2.7249212221777306, 0.2762925796411284]], [[-3.4715782802404567, 0.6723628889324178], [-2.6749766971671414, 2.573509573529996], [-2.9330954158764904, 0.2526423626309662], [-2.8237942655813058, 0.1774195362375537]], [[-2.4533608815543113, 1.6905802876185634], [-0.3555991951157389, 4.892887075581399], [-2.8924972537953453, 0.2932405247121108], [-2.7032236093826265, 0.2979901924362329]], [[-0.6704622164548715, 3.473478952718003], [-2.389612949887905, 2.8588733208092325], [-0.39014683396417205, 2.7955909445432843], [-0.2175

# variations
**Choose t diffrent random center of each cluster's center in shrinked sampling interval neghborhood**

In [67]:
def variation(candidates_array):
    for candidate in candidates_array:
        variations = []
        for i in range(0,variations_count):
            Second_layer = []
            for j in candidate.sampling_interval:
                Third_layer = []
                for k in range(0,len(j)):
                    #print(k)
                    Third_layer.append(random.uniform(j[k][0],j[k][1]))
                Second_layer.append(Third_layer)
            variations.append(Second_layer)
        candidate.variation = variations    
variation(New_candidates_array)
print(New_candidates_array[3].variation[1])

[[-2.0398444819827732, -0.4550176408395079, -1.250994360920871, 0.6402265260771531], [2.06277945791099, -1.2507999997372146, 2.2318877398523527, 0.3175815038229919], [1.2171396911965529, 3.1182068069664832, -2.684777957472417, -1.5911696689656059], [-0.3765165789494893, 0.9802320619304208, -2.8390383426346775, -2.30633628633306], [3.3775267800797932, 0.5468781286015747, 2.2932333147270945, 0.6758356106631673], [2.0232726026912773, -0.7022100558382074, -1.0147784835780904, 0.14553912672493796]]
